# AI as Judge

[G-Eval](https://deepeval.com/docs/metrics-llm-evals) is a framework that uses LLM as a judge to evaluate LLM outputs. The evaluation can be based on any criteria. G-Eval is implemented by a library called [DeepEval](https://deepeval.com/) which includes a broader set of tests.


In [1]:
%load_ext dotenv
%dotenv ../../05_src/.secrets

In [2]:
from openai import OpenAI
import os

document_folder = "../../05_src/documents/"
blue_cross_file = "the_blue_cross.txt"
file_path = os.path.join(document_folder, blue_cross_file)

with open(file_path, "r", encoding="utf-8") as f:
    blue_cross_text = f.read()

In [3]:
instructions = "You are an helpful assistant that summarizes works of fiction with a quirky and bubbly approach."
PROMPT = """
    Summarize the following story in at most four paragraphs. Please include all key characters and plot points.
    <story>
    {story}
    </story>
    In addition to the summary, add an introduction paragraph where you greet the reader and a conclusion where you share an opinion about the story.
"""

In [4]:
client = OpenAI()
response = client.responses.create(
    model="gpt-4o-mini",
    instructions=instructions,
    input=[
        {"role": "user", 
         "content": PROMPT.format(story=blue_cross_text)}
    ],
    temperature=1.2
)

In [5]:
response.output_text

"Hello there, adventurous reader! 🌟 Buckle up for a whimsical ride through the clever corners of 'The Blue Cross,' where a detective board collides with a master criminal, and secrets flutter on the wind like fallen leaves—let's dive in!\n\nIn this delightful tale, we meet Aristide Valentin, the shrewd head of the Paris police, who’s on an urgent mission to capture the notorious criminal Flambeau at the bustling Eucharistic Congress in London. Armed with only his snazzy attire, a revolver under his jacket, and a quick-thinking mind, Valentin sets out in pursuit of the giant criminal whose doodles of daring have made him a towering figure in the world of crime. Balancing official decorum with worldly charm, Valentin navigates through twisty paths, searching for Flambeau among ordinary folks and listening for whispers that prick sharper than daggers.\n\nNow, swirl with those butterflies of anticipation as Flambeau, the charming trickster, operates under an alias, all while plotting to sn

# Answer Relevancy

The answer relevancy metric evaluates how relevant the actual output of the LLM app is compared to the provided input. This metric is self-explaining in the sense that the output includes a reason for the metric score.

The metric is calculated as:

$$
AnswerRelevancy=\frac{NumberRelevantStatements}{TotalStatements}
$$

Reference: [Answer Relevancy](https://deepeval.com/docs/metrics-answer-relevancy). 

In [6]:
from deepeval import evaluate
from deepeval.metrics import AnswerRelevancyMetric
from deepeval.test_case import LLMTestCase

metric = AnswerRelevancyMetric(
    threshold=0.7,
    model="gpt-4o-mini",
    include_reason=True
)

test_case = LLMTestCase(
    input=PROMPT.format(story=blue_cross_text),
    actual_output=response.output_text
)

In [7]:
metric.measure(test_case)

Output()

0.9230769230769231

In [8]:
print(metric.score,metric.reason)

0.9230769230769231 The score is 0.92 because while the summary effectively captures the key characters and plot points of the story, it incorrectly states that Flambeau operates under an alias, which is not true as he is known by his real name. This minor inaccuracy prevents the score from being higher, but the overall quality of the summary and the inclusion of essential details justify the current score.


# Other Metrics

Other useful metric functions include:

+ [Faithfulness](https://deepeval.com/docs/metrics-faithfulness): evaluates whether the `actual_output` factually aligns with the contents of  `retrieval_context`. 
+ [Contextual Precision](https://deepeval.com/docs/metrics-contextual-precision): evaluates whether nodes in your `retrieval_context` that are relevant to the given input are ranked higher than irrelevant ones. 
+ [Contextual Recall](https://deepeval.com/docs/metrics-contextual-recall): evaluates the extent of which the retrieval_context aligns with the expected_output. 
+ [Contextual Relevancy](https://deepeval.com/docs/metrics-contextual-relevancy): evaluates the overall relevance of the information presented in your retrieval_context for a given input. 

# G-Eval

[G-Eval](https://deepeval.com/docs/metrics-llm-evals) is a framework that uses LLM-as-a-judge with chain-of-thoughts (CoT) to evaluate LLM outputs based on ANY custom criteria. The G-Eval metric is the most versatile type of metric deepeval offers.

In [9]:
instructions = "You are an helpful assistant that specializes in works of fiction."
PROMPT = """
    Based on the story below, answer the question provided.
    <story>
    {story}
    </story>
    <question>
    Who is the main antagonist in the story and what motivates their actions?
    </question>
"""

In [10]:
client = OpenAI()
response = client.responses.create(
    model="gpt-4o",
    instructions=instructions,
    input=[
        {"role": "user", 
         "content": PROMPT.format(story=blue_cross_text)}
    ],
    temperature=0.7
)

In [11]:
response.output_text

'The main antagonist in the story is Flambeau, a notorious and ingenious criminal. His actions are motivated by his desire to commit high-profile thefts, such as stealing the valuable sapphire cross in this story. Flambeau is known for his creative and daring criminal exploits, often employing clever disguises and elaborate schemes to achieve his goals. His motivation is primarily driven by the thrill of the crime and the challenge it presents, as well as the potential for significant gain.'

## Evaluation Criteria

The most straightforward way to establish a metric is by using a single criteria.

In [12]:
from deepeval.metrics import GEval
from deepeval.test_case import LLMTestCaseParams

correctness_metric = GEval(
    name="Correctness",
    criteria="Determine whether the actual output is factually correct based on the context.",
    evaluation_params=[LLMTestCaseParams.INPUT, LLMTestCaseParams.ACTUAL_OUTPUT],
)

In [13]:
test_case = LLMTestCase(
    input=PROMPT.format(story=blue_cross_text),
    actual_output=response.output_text
)
evaluate(test_cases=[test_case], metrics=[correctness_metric])

✨ You're running DeepEval's latest Correctness [GEval] Metric! (using gpt-4.1, strict=False, async_mode=True)...

Output()

PermissionDeniedError: Error code: 403 - {'error': {'message': 'Project `proj_azcDlGrYmDy6eV8yO8hoT2pv` does not have access to model `gpt-4.1`', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}

## Evaluation Steps 

G-Eval is flexible in many ways: notice that we can establish an evaluation criteria or a set of evaluation steps, that can help in guiding the model to follow specific steps to perform the evaluation.

In [ ]:
...

correctness_metric = GEval(
    name="Correctness",
    evaluation_steps=[
        "Check whether the facts in 'actual output' contradicts any facts in 'input'",
        "You should also heavily penalize omission of detail",
        "Vague language, or contradicting OPINIONS, are not OK"
    ],
    evaluation_params=[LLMTestCaseParams.INPUT, LLMTestCaseParams.ACTUAL_OUTPUT],
)

In [ ]:
test_case = LLMTestCase(
    input=PROMPT.format(story=blue_cross_text),
    actual_output=response.output_text
)
evaluate(test_cases=[test_case], metrics=[correctness_metric])